# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [1]:
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "voc-1838082046126677398923067f2bebb053119.79748012"

In [2]:
import pandas as pd

# Load the CSV file
file_path = 'data/character_descriptions.csv'
character_data = pd.read_csv(file_path)

# Function to combine columns into a single descriptive text
def combine_columns(row):
    # Combine the relevant columns into a single descriptive text
    text = f"{row['Name']} is a {row['Description']} This character appears in a {row['Medium']} set in {row['Setting']}."
    return text

# Apply the function to each row in the dataframe
character_data['text'] = character_data.apply(combine_columns, axis=1)

# Create a new dataframe with only the 'text' column
combined_df = character_data[['text']]

# Display the resulting dataframe
combined_df.head(30)
#print(f"Total characters: {len(combined_df)}")

,text
0,"Emily is a A young woman in her early 20s, Emi..."
1,"Jack is a A middle-aged man in his 40s, Jack i..."
2,"Alice is a A woman in her late 30s, Alice is a..."
3,"Tom is a A man in his 50s, Tom is a retired so..."
4,"Sarah is a A woman in her mid-20s, Sarah is a ..."
5,"George is a A man in his early 30s, George is ..."
6,"Rachel is a A woman in her late 20s, Rachel is..."
7,"John is a A man in his 60s, John is a retired ..."
8,Maria is a A middle-aged Latina woman in her 4...
9,Caleb is a A young African American man in his...


In [3]:
import random

# Randomly select two rows from the dataframe
sampled_rows = combined_df.sample(2).reset_index(drop=True)

# Extract the names of the characters for the questions
character_1 = sampled_rows.iloc[0]['text'].split()[0]
character_2 = sampled_rows.iloc[1]['text'].split()[0]

# Generate two questions based on the selected characters
prompt1 = f"What is Malvolio's profession?"
prompt2 = f"In what setting does karma's story take place?"

print(f'Prompt1: {prompt1}')

answer1 = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt1,
    max_tokens=150
)["choices"][0]["text"].strip()
print(answer1)

print(f'Prompt2: {prompt2}')

answer2 = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt2,
    max_tokens=150
)["choices"][0]["text"].strip()
print(answer2)

EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(combined_df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=combined_df.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )
    
    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
combined_df["embeddings"] = embeddings
combined_df

# Save the csv file with embeddings
combined_df.to_csv('character_descriptions_with_embeddings.csv')

len(combined_df['embeddings'][0])
!ls

Prompt1: What is Malvolio's profession?
Malvolio's profession is steward or the chief servant of Lady Olivia in William Shakespeare's play "Twelfth Night."
Prompt2: In what setting does karma's story take place?
Karma's story takes place in a variety of settings as it follows her journey through different lifetimes and reincarnations. However, the main setting of the story is often depicted as a sort of spiritual realm or afterlife, where karma and other spiritual beings exist and oversee the cycle of rebirth and retribution. Other settings include different time periods and locations on Earth, where karma's various incarnations live and learn important lessons.


/tmp/ipykernel_451/2080017279.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df["embeddings"] = embeddings


Custom_Chatbot_Documented_FINAL.ipynb	    data
character_descriptions_with_embeddings.csv  project.ipynb


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [4]:
import numpy as np
import pandas as pd
# import openai
# openai.api_base = "https://openai.vocareum.com/v1"
# openai.api_key = "YOUR API KEY"

# load the df with embeddings
file_path = 'character_descriptions_with_embeddings.csv'
df = pd.read_csv(file_path, index_col= 0)
df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)

from openai.embeddings_utils import get_embedding, distances_from_embeddings

def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy

In [5]:
df['embeddings'][0].shape

(1536,)

In [6]:
# do this for question 1
question1 = "What is Malvolio's profession?"
sorted_df1 = get_rows_sorted_by_relevance(question1, df)
sorted_df1.head(5)

,text,embeddings,distances
42,Malvolio is a A pompous and self-righteous ste...,"[-0.015128765255212784, -0.034387096762657166,...",0.112696
45,Bianca is a Lady Olivia's cunning and quick-wi...,"[-0.015198267996311188, -0.025840982794761658,...",0.162375
43,Viola is a A plucky and resourceful young woma...,"[-0.010260283946990967, -0.041502270847558975,...",0.169584
40,Lady Olivia is a A wealthy and beautiful noble...,"[-0.019489478319883347, -0.023335402831435204,...",0.185614
41,Sir Toby Belch is a A drunken and lecherous kn...,"[-0.0016695825615897775, -0.032771751284599304...",0.192171


In [7]:
# do this for question 2
question2 = "In what setting does Karma's story take place?"
sorted_df2 = get_rows_sorted_by_relevance(question2, df)
sorted_df2.head(5)

,text,embeddings,distances
24,"Karma is a A chameleon-like performer, Karma i...","[-0.006748127285391092, -0.020624373108148575,...",0.185979
11,"Sonya is a A white woman in her late 20s, Sony...","[0.002259109867736697, -0.02683749422430992, -...",0.232009
4,"Sarah is a A woman in her mid-20s, Sarah is a ...","[-0.017505977302789688, -0.02165350690484047, ...",0.235176
3,"Tom is a A man in his 50s, Tom is a retired so...","[0.014915194362401962, -0.01043409388512373, -...",0.239343
8,Maria is a A middle-aged Latina woman in her 4...,"[-0.009636449627578259, -0.0114671029150486, -...",0.240752


In [8]:
import tiktoken

prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""


def create_prompt(question, df, max_token_count, prompt_template=prompt_template):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")

    
    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n----------------------------\n\n".join(context), question)

# create prompt for question 1
max_token_count = 300
print(create_prompt(question1, df, max_token_count))


# create prompt for question 2
max_token_count = 300
print(create_prompt(question2, df, max_token_count))

COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    
    prompt = create_prompt(question, df, max_prompt_tokens)
    
    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""
    
    # answer question 1
answer1 = answer_question(question1, df)
print(answer1)

answer2 = answer_question(question2, df)
print(answer2)




Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

Malvolio is a A pompous and self-righteous steward in Lady Olivia's household. Malvolio is humorless and uptight, and is often the target of Sir Toby Belch's pranks. He is secretly in love with Lady Olivia and harbors dreams of marrying her. This character appears in a Play set in Ancient Greece.

----------------------------

Bianca is a Lady Olivia's cunning and quick-witted maid. Bianca is a master of mischief and pranks, and often collaborates with Sir Toby Belch to torment Malvolio. She is also secretly in love with Sir Toby. This character appears in a Play set in Ancient Greece.

----------------------------

Viola is a A plucky and resourceful young woman who is shipwrecked on the coast of Illyria. Viola disguises herself as a man, taking on the identity of Cesario, and becomes a servant in Duke Orsino's household. She develops feelings for

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [9]:
question1 = "What is Tom's profession?"

answer1 = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=question1,
    max_tokens=150
)["choices"][0]["text"].strip()
print(answer1)

As an AI, I am not able to access personal information. I cannot confirm Tom's profession.


In [10]:
custom_chatbot_anwer1 = answer_question(question1, df)
print(custom_chatbot_anwer1)

Retired soldier


### Question 2

In [11]:
question2 = "In what setting does Thomas's story take place?"

answer2 = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=question2,
    max_tokens=150
)["choices"][0]["text"].strip()
print(answer2)

Thomas's story takes place in a small town in Southern California during the summer.


In [12]:
custom_chatbot_anwer2 = answer_question(question2, df)
print(custom_chatbot_anwer2)

A Sitcom set in USA.
